In [1]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter as sg_filter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil

from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import correct_predictor_columns, correct_timestamps, resample_or_rolling_stats, input_data_split
from wattile.time_processing import add_processed_time_columns
from wattile.models.liangs_model import main as liangs_model
PROJECT_DIRECTORY = Path().resolve().parent.parent

### reading configs

In [2]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# # main configs file
# with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################
# code testing configs file
with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
    configs = json.load(f)
##################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["exp_dir"] = str(exp_dir)
configs["num_epochs"] = 10
configs["data_dir"] = str(PROJECT_DIRECTORY / "tests" / "data")

configs

{'building': 'Synthetic Site',
 'target_var': 'Synthetic Site Electricity Main Total Power',
 'start_time': '2018-01-01T00:00:00-07:00',
 'end_time': '2022-01-01T00:00:00-07:00',
 'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
  'Synthetic Weather Station Diffuse Horizontal Irradiance',
  'Synthetic Weather Station Direct Normal Irradiance',
  'Synthetic Weather Station Dry Bulb Temperature',
  'Synthetic Weather Station Global Horizontal Irradiance',
  'Synthetic Weather Station Relative Humidity',
  'Synthetic Weather Station Wind Speed'],
 'arch_version': 4,
 'exp_id': 'Debugging_for_rolling_stats',
 'arch_type': 'RNN',
 'arch_type_variant': 'lstm',
 'transformation_method': 'minmaxscale',
 'train_batch_size': 5,
 'val_batch_size': 1,
 'convert_csvs': False,
 'exp_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\notebooks\\exp_dir',
 'data_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\tests\\data'

### override config if necessary

In [3]:
# configs["feat_stats"]["window_width"] = '15min'
# configs["feat_stats"]["window_increment"] = '15min'

### reading data

In [4]:
site = configs['building']

In [5]:
data = read_dataset_from_file(configs)
data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


### feature extraction

In [6]:
# assert we have the correct columns and order them
data = correct_predictor_columns(configs, data)

# sort and trim data specified time period
data = correct_timestamps(configs, data)

# Add time-based features
data = add_processed_time_columns(data, configs)

# Add statistics features
data = resample_or_rolling_stats(data, configs)

data

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,DOW_binary_reg_0_min,...,DOW_binary_reg_0_mean,DOW_binary_reg_1_mean,DOW_binary_reg_2_mean,DOW_binary_reg_3_mean,DOW_binary_reg_4_mean,DOW_binary_reg_5_mean,DOW_binary_reg_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-2.588190e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,104.800003
2021-12-01 07:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-3.214395e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,103.650002
2021-12-01 07:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-3.826834e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.050003
2021-12-01 07:45:00+00:00,16.614972,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-4.422887e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.349998
2021-12-01 08:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-5.000000e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,102.949997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:00:00+00:00,11.961447,-1.673728,-1.211398,41.388802,-1.909901,25.160000,0.000000,0.998135,-1.608123e-16,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,104.668396
2021-12-08 06:15:00+00:00,11.008480,-1.489343,-0.504314,41.646198,-1.936419,24.240000,0.000000,0.997859,-6.540313e-02,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,102.709602
2021-12-08 06:30:00+00:00,10.851442,-1.571301,-0.655083,42.024200,-1.834018,23.760000,0.000000,0.991445,-1.305262e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,105.972198


In [7]:
data_wTS = data.copy()

# =======================================
# testing
# =======================================

In [8]:
# reading configuration settings
# configs["window_source_size"] = 12
# configs["window_target_size"] = 3
# configs["normalization"] = False
configs["S2S_window"] = {
  "window_width_source": 12,
  "window_width_target": 3,
  "normalization": False
 }

 TODOs:
 
 - `roll_full_data_s2s` output to `data`
 - use above `data` as input to `input_data_split`
 - connect `val_df_target` properly to results plot
 - `usage_actual = data[configs["target_var"]]`
 - test data converted to 3D with every interval instead of interval with window_target_size

### original

In [9]:
dataset = data_wTS.astype(np.float32).copy()

####################################################################################
# roll_full_data_s2s
####################################################################################
normalization = configs["S2S_window"]["normalization"]
window_source_size = configs["S2S_window"]["window_width_source"]
window_target_size = configs["S2S_window"]["window_width_target"]

# normalization
if normalization:
    print("Transforming data to 0 mean and unit var")
    MU = dataset.mean(0)  # 0 means take the mean of the column
    dataset = dataset - MU
    STD = dataset.std(0)  # same with std here
    dataset = dataset / STD

# initialize lists
data_predictor = []
data_target = []

tic = time.time()

#####################################################################################
# create rolling window data for both predictor and target and for training data set
#####################################################################################
data = dataset.values
idxs = np.arange(
    0,
    data.shape[0] - (window_source_size + window_target_size)
)
for idx in idxs:
    data_predictor.append(
        data[idx : idx + window_source_size].reshape(
            (1, window_source_size, data.shape[1])
        )
    )
    data_target.append(
        data[
            idx
            + window_source_size : idx
            + window_source_size
            + window_target_size,
            -1,
        ].reshape((1, window_target_size, 1))
    )

# convert to numpy array
data_predictor = np.concatenate(data_predictor, axis=0)
data_target = np.concatenate(data_target, axis=0) 

# combining 3D predictor and target data into dictionary
data = {}
data['predictor'] = data_predictor
data['target'] = data_target
#####################################################################################

toc = time.time()
print("time elapsed = {}".format(toc-tic))

time elapsed = 0.004992961883544922


In [10]:
data

{'predictor': array([[[ 1.54137335e+01, -1.52585006e+00, -5.19907009e-03, ...,
          -4.93775547e-01,  8.69589388e-01,  1.04800003e+02],
         [ 1.47829657e+01, -1.49622595e+00, -5.40706813e-01, ...,
          -4.93775547e-01,  8.69589388e-01,  1.03650002e+02],
         [ 1.54748497e+01, -1.37941206e+00, -1.74170601e+00, ...,
          -4.93775547e-01,  8.69589388e-01,  9.80500031e+01],
         ...,
         [ 1.70106049e+01, -1.74843705e+00, -1.92366803e+00, ...,
          -4.93775547e-01,  8.69589388e-01,  9.79499969e+01],
         [ 1.99186363e+01, -1.78446996e+00, -2.07443309e+00, ...,
          -4.93775547e-01,  8.69589388e-01,  1.02000000e+02],
         [ 2.06585693e+01, -1.50733495e+00, -8.31857800e-01, ...,
          -4.93775547e-01,  8.69589388e-01,  9.95000000e+01]],
 
        [[ 1.47829657e+01, -1.49622595e+00, -5.40706813e-01, ...,
          -4.93775547e-01,  8.69589388e-01,  1.03650002e+02],
         [ 1.54748497e+01, -1.37941206e+00, -1.74170601e+00, ...,
        

In [11]:
data_predictor.shape

(658, 12, 55)

In [12]:
data_target.shape

(658, 3, 1)

### modified

In [13]:
# reading configuration settings
configs["S2S_window"] = {
  "window_width_source": "180min",
  "window_width_target": "45min",
  "normalization": False
 }

In [14]:
data = data_wTS.astype(np.float32).copy()

tic = time.time()

#####################################################################################
# create rolling window data for both predictor and target and for training data set
#####################################################################################

# setting configuration parameters
normalization = configs["S2S_window"]["normalization"]
window_width_source = configs["S2S_window"]["window_width_source"]
window_width_target = configs["S2S_window"]["window_width_target"]
resample_interval = configs["resample_interval"]
target_var = configs["target_var"]

# normalization
if normalization:
    print("Transforming data to 0 mean and unit var")
    MU = data.mean(0)  # 0 means take the mean of the column
    data = data - MU
    STD = data.std(0)  # same with std here
    data = data / STD

# initialize lists
data_predictor = []
data_target = []

# calculate number of rows based on window size defined by time
window_source_size_count = int(
    pd.Timedelta(window_width_source) / pd.Timedelta(resample_interval)
)
window_target_size_count = int(
    pd.Timedelta(window_width_target) / pd.Timedelta(resample_interval)
)

# set aside timeindex
timestamp = data.iloc[
    : -(window_source_size_count + window_target_size_count - 1), :
].index

# create 3D predictor data
data_shifted_predictor = data.iloc[:-window_target_size_count, :]
for window in data_shifted_predictor.rolling(window=window_width_source):
    if window.shape[0] == window_source_size_count:
        data_predictor.append(
            window.values.reshape(
                (1, window_source_size_count, data_shifted_predictor.shape[1])
            )
        )

# create 3D target data
data_shifted_target = data.loc[
    data.index >= data.shift(freq=window_width_source).index[0], :
][target_var]
for window in data_shifted_target.rolling(window=window_width_target):
    if window.shape[0] == window_target_size_count:
        data_target.append(window.values.reshape((1, window_target_size_count, 1)))

# reshape data dimension
data_predictor = np.concatenate(np.array(data_predictor), axis=0)
data_target = np.concatenate(np.array(data_target), axis=0)

# combine 3D predictor and target data into dictionary
data = {}
data["predictor"] = data_predictor
data["target"] = data_target
data["timestamp"] = timestamp
#####################################################################################

toc = time.time()
print("time elapsed = {}".format(toc-tic))

time elapsed = 0.10199999809265137


In [15]:
data_predictor.shape

(659, 12, 55)

In [16]:
data_target.shape

(659, 3, 1)

In [17]:
timestamp.shape

(659,)